# Training + Eval Runs

In [1]:
## GENERATE SINGLE-TASK FT DATA

import os
os.makedirs("ft_data", exist_ok=True)

bases = [8, 9, 10]
train_n_digits = [2]
train_types = ["ft_lora"]

command_file = "train_commands.sh"


## datagen code
for base in bases:
    for n_digits in train_n_digits:    
        command = f"python sample_pipe.py --base {base} --n_digits {2} --n_samples {1000}"
        with open(command_file, "a") as f:
            f.write(command + "\n")

In [9]:
## GENERATE LORA FT COMMANDS

## training code
for train_type in train_types:
    for base in bases:
        for n_digits in train_n_digits:
            command = f"python {train_type}.py --base {base} --model_path unsloth/Phi-4 {cot} --data_file ft_data/data_ft_{base}_{n_digits}.txt"
            with open(command_file, "a") as f:
                f.write(command + "\n")


In [7]:
## GENERATE OA FT COMMANDS

for train_type in train_types:
    for base in bases:
        for n_digits in train_n_digits:
            command = f"python {train_type}.py --base {base} --model_path gpt-4o-2024-08-06 {cot} --data_file ft_data/data_ft_{base}_{n_digits}.txt"
            with open(command_file, "a") as f:
                f.write(command + "\n")

In [ ]:
## RUN TRAINING

#bash train_commands.sh 

In [8]:
## COMPUTE EVALS

eval_checkpoints = [63]
ks = [1, 2, 3, 4, 5, 6, 8, 10, 12, 16]
command_file = "eval_commands.sh"
                
## sft evals
models = os.listdir("outputs")
for checkpoint in eval_checkpoints:
    for base in bases:
        for n_digits in [2, 3, 4]:
            for model in models:
                command = f"python eval_pipe.py --base {base} --model_name outputs/{model}/checkpoint-{checkpoint} --size 300 --n_digits {n_digits}"
                with open(command_file, "a") as f:
                    f.write(command + "\n")
                    f.write("rm output.txt" + "\n")
# baseline evals
for cot in [True, False]:
    for base in bases:
        for n_digits in [2, 3, 4]:
            command = f"python eval_pipe.py --base {base} --model_name unsloth/Phi-4 --size 300 --n_digits {n_digits}"
            with open(command_file, "a") as f:
                f.write(command + "\n")
                f.write("rm output.txt" + "\n")

# icl evals
for k in ks:
    for base in bases:
        for n_digits in [2]:
            command = f"python eval_pipe.py --base {base} --model_name unsloth/Phi-4 --size 300 --n_digits {n_digits} --n_shots {k}"
            with open(command_file, "a") as f:
                f.write(command + "\n")
                f.write("rm output.txt" + "\n")

In [12]:
## GENERATE OA EVAL COMMANDS


with open("outputs/oai_models", "r") as f:
    models = f.read().split("\n")
    models = dict([(model.split(": ")[0], model.split(": ")[-1]) for model in models])
    code2model = dict([(model.split(": ")[-1], model.split(": ")[0]) for model in models])


models = {'test_run_8_cot_True_n_digits_2': "ft:gpt-4o-2024-08-06:kayafamily::BhAG4cP5",
 'test_run_8_cot_False_n_digits_2': "ft:gpt-4o-2024-08-06:kayafamily::BhAFt3u6",
 'test_run_9_cot_True_n_digits_2': "ft:gpt-4o-2024-08-06:kayafamily::BhAXZdJQ",
 'test_run_9_cot_False_n_digits_2': "ft:gpt-4o-2024-08-06:kayafamily::BhO7qm7Y",
 'test_run_10_cot_True_n_digits_2': "ft:gpt-4o-2024-08-06:kayafamily::BhO5LNSi",
 'test_run_10_cot_False_n_digits_2': "ft:gpt-4o-2024-08-06:kayafamily::BhOATqLQ"}

code2model = {"ft:gpt-4o-2024-08-06:kayafamily::BhAG4cP5":'test_run_8_cot_True_n_digits_2',
 'ft:gpt-4o-2024-08-06:kayafamily::BhAFt3u6': 'test_run_8_cot_False_n_digits_2',
 'ft:gpt-4o-2024-08-06:kayafamily::BhAXZdJQ': 'test_run_9_cot_True_n_digits_2',
 'ft:gpt-4o-2024-08-06:kayafamily::BhO7qm7Y': 'test_run_9_cot_False_n_digits_2',
 'ft:gpt-4o-2024-08-06:kayafamily::BhO5LNSi': 'test_run_10_cot_True_n_digits_2',
 "ft:gpt-4o-2024-08-06:kayafamily::BhOATqLQ":'test_run_10_cot_False_n_digits_2'}



In [ ]:
for base in bases:
    for n_digits in [2, 3, 4]:
        for name, model in models.items():
            command = f"python eval_pipe.py --base {base} --model_name {model} --size 300 --n_digits {n_digits}"
            with open(command_file, "a") as f:
                f.write(command + "\n")
                f.write("rm output.txt" + "\n")


for cot in [True, False]:
    for base in bases:
        for n_digits in [2, 3, 4]:
            command = f"python eval_pipe.py --base {base} --model_name gpt-4o-2024-08-06 --size 300 --n_digits {n_digits}"
            with open(command_file, "a") as f:
                f.write(command + "\n")
                f.write("rm output.txt" + "\n")

In [ ]:
## RUN EVALS

#bash eval_commands.sh 


# Analysis